In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Flowers-102 dataset
(ds_train, ds_test), ds_info = tfds.load('tf_flowers', split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=102)  # 102 classes in Flowers-102 dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (64, 224, 224, 3)
Labels shape: (64, 102)
Images shape: (64, 224, 224, 3)
Labels shape: (64, 102)


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(102, activation='softmax'))  # Change the number of units and activation

# Assuming ds_train is your full dataset
# You need to have a reasonable amount of data for both training and validation for this to be effective

# Define the ratio of data to be used for validation
validation_split = 0.2

# Calculate the number of samples to be used for validation
validation_samples = int(validation_split * len(ds_train))

# Calculate the number of samples to be used for training
training_samples = len(ds_train) - validation_samples

# Split the data
ds_validation = ds_train.take(validation_samples)
ds_train = ds_train.skip(validation_samples)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model with early stopping
model.fit(ds_train, batch_size=32, epochs=50, validation_data=ds_validation)


Epoch 1/50
30/30 [==============================] - 22s 438ms/step - loss: 1.0342 - accuracy: 0.7385 - val_loss: 136.9716 - val_accuracy: 0.2812
Epoch 2/50
30/30 [==============================] - 11s 354ms/step - loss: 0.4717 - accuracy: 0.8426 - val_loss: 412.1770 - val_accuracy: 0.1875
Epoch 3/50
30/30 [==============================] - 11s 361ms/step - loss: 0.3737 - accuracy: 0.8808 - val_loss: 37.8941 - val_accuracy: 0.2455
Epoch 4/50
30/30 [==============================] - 12s 367ms/step - loss: 0.2664 - accuracy: 0.9121 - val_loss: 14.1354 - val_accuracy: 0.2478
Epoch 5/50
30/30 [==============================] - 11s 365ms/step - loss: 0.1593 - accuracy: 0.9440 - val_loss: 2.2399 - val_accuracy: 0.2656
Epoch 6/50
30/30 [==============================] - 12s 364ms/step - loss: 0.1350 - accuracy: 0.9514 - val_loss: 4.1508 - val_accuracy: 0.2545
Epoch 7/50
30/30 [==============================] - 11s 354ms/step - loss: 0.1153 - accuracy: 0.9634 - val_loss: 3.4847 - val_accuracy: 

In [6]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


12/12 [==============================] - 2s 121ms/step
Test Accuracy: 0.8651226158038147
Accuracy: 0.8651226158038147
F1 Score: 0.8662971378516718


In [ ]:
from tensorflow import keras
model = keras.models.load_model(r"C:\Users\shaif\Downloads\Flowers102_224_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 102  # Update with the actual number of classes in your target data
x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)

# Define the ratio of data to be used for validation
validation_split = 0.2

# Calculate the number of samples to be used for validation
validation_samples = int(validation_split * len(ds_train))

# Calculate the number of samples to be used for training
training_samples = len(ds_train) - validation_samples

# Split the data
ds_validation = ds_train.take(validation_samples)
ds_train = ds_train.skip(validation_samples)

# Compile the model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
#early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

# Train the model with early stopping
new_model.fit(ds_train, batch_size=32, epochs=50, validation_data=ds_validation)

Epoch 1/50
24/24 [==============================] - 18s 436ms/step - loss: 1.8336 - accuracy: 0.4136 - val_loss: 28095.9141 - val_accuracy: 0.1693
Epoch 2/50
24/24 [==============================] - 9s 359ms/step - loss: 1.1349 - accuracy: 0.5641 - val_loss: 27241.4863 - val_accuracy: 0.1667
Epoch 3/50
24/24 [==============================] - 10s 376ms/step - loss: 1.0633 - accuracy: 0.6093 - val_loss: 936.0186 - val_accuracy: 0.1693
Epoch 4/50
24/24 [==============================] - 10s 390ms/step - loss: 1.0390 - accuracy: 0.6185 - val_loss: 339.1466 - val_accuracy: 0.1849
Epoch 5/50
24/24 [==============================] - 9s 370ms/step - loss: 0.9640 - accuracy: 0.6361 - val_loss: 111.5097 - val_accuracy: 0.1276
Epoch 6/50
24/24 [==============================] - 10s 380ms/step - loss: 0.9004 - accuracy: 0.6538 - val_loss: 4.7661 - val_accuracy: 0.3750
Epoch 7/50
24/24 [==============================] - 10s 385ms/step - loss: 0.8660 - accuracy: 0.6695 - val_loss: 7.9065 - val_accu

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

# Make predictions on the test dataset
predictions = new_model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


In [7]:
pip install protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.
